In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Add
import tensorflow.keras.backend as K
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 11GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=11264)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # vector input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # label input
    in_label = Input(shape=(1,))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((image_shape[0], image_shape[1], image_shape[2]))(li)
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image, li])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(latent_dim)(li)
    li = Reshape([latent_dim])(li)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    b = Dense(8000)(b)
    projection = Reshape((40, 200))(b)

    m = Concatenate()([gen, li])
    n_nodes = (vector_shape[0] * vector_shape[1])
    m = Dense(n_nodes)(m)
    m = Reshape([vector_shape[0], vector_shape[1]])(m)
    d = LSTM(200, activation='tanh', return_sequences=True)(m)

    # Layer normalization 1.
    x1 = LayerNormalization(epsilon=1e-6)(projection)
    # Create a multi-head attention layer.
    attention_output = MultiHeadAttention(
        num_heads=4, key_dim=200, dropout=0.1
    )(d, x1)
    # Skip connection 1.
    x2 = Add()([attention_output, x1])
    # Reshape to vector size

    logits = Dense(3)(x2)
    # encoder-decoder LSTM model
    # d = LSTM(200, activation='relu')(m)
    # d = Flatten()(d)
    # d = Dense(512)(d)
    # attention = MultiHeadAttention()([b, d])
    # p = Dense(120)(attention)
    # output
    out_layer = Reshape((40, 3))(logits)

    # define model input & output
    model = Model([in_image, in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat, in_label])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out, in_label])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat, in_label], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
	return [X1, X2, X3-1]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	img, vec, label = dataset
	# choose random instances
	ix = randint(0, img.shape[0], n_samples)
	# retrieve selected images
	X1, X2, X3 = img[ix], vec[ix], label[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2, X3], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, label, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input, label])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
def generate_drvact_text(drvact):
    text = '[warnings] drvact label is not defined ...'
    if (drvact == 1):
        text = 'Go'
    elif (drvact == 2):
        text = 'Turn Left'
    elif (drvact == 3):
        text = 'Turn Right'
    elif (drvact == 4):
        text = 'U-turn'
    elif (drvact == 5):
        text = 'Left LC'
    elif (drvact == 6):
        text = 'Right LC'
    elif (drvact == 7):
        text = 'Avoidance'
    return text

In [13]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_real_img, X_real_vec, X_label], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fake_vec, _ = generate_fake_samples(g_model, X_real_img, X_label, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_real_img = (X_real_img + 1) / 2.0
    pyplot.figure(figsize=(32.0, 20.0))
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_real_img[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_fake_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_real_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    filename3 = 'd_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('>Saved: %s, %s and %s' % (filename1, filename2, filename3))

In [14]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=100, batch_size=30):
    # iteration number to output checkpoint
    display_iter = 1000
    # calculate the number of batches
    n_batch = int(len(dataset[0]) / batch_size)
    # calculate the number of training iterations
    n_steps = n_batch * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_real_img, X_real_vec, X_label], y_real = generate_real_samples(dataset, batch_size)
        # generate a batch of fake samples
        X_fake_vec, y_fake = generate_fake_samples(g_model, X_real_img, X_label, batch_size)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_real_img, X_real_vec, X_label], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_real_img, X_fake_vec, X_label], y_fake)
        X_lat = generate_latent_points(latent_dim, batch_size)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_real_img, X_lat, X_label], [y_real, X_real_vec])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        #if (i+1) % int(bat_per_epo) == 0:
        if (i+1) % int(display_iter) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')

In [15]:
%%time
# load image data
dataset = load_real_samples('./dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape, dataset[2].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3) (16805,)
CPU times: user 22.8 s, sys: 774 ms, total: 23.6 s
Wall time: 23.6 s


In [16]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
# %%
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)
# %%
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [17]:
#see model summary
print(g_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 64) 3136        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 64, 128)  131200      leaky_re_lu_7[0][0]              
____________________________________________________________________________________________

In [18]:
# train model
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.456] d2[1.831] g[419.666]
>2, d1[0.095] d2[0.578] g[397.502]
>3, d1[0.164] d2[0.484] g[376.791]
>4, d1[0.178] d2[0.515] g[292.160]
>5, d1[0.236] d2[0.530] g[227.611]
>6, d1[0.198] d2[0.622] g[217.150]
>7, d1[0.217] d2[0.522] g[213.636]
>8, d1[0.262] d2[0.389] g[228.634]
>9, d1[0.407] d2[0.454] g[187.160]
>10, d1[0.284] d2[0.385] g[202.685]
>11, d1[0.491] d2[0.347] g[177.065]
>12, d1[0.356] d2[0.372] g[213.052]
>13, d1[0.376] d2[0.374] g[223.111]
>14, d1[0.287] d2[0.307] g[243.429]
>15, d1[0.439] d2[0.389] g[183.682]
>16, d1[0.433] d2[0.303] g[172.077]
>17, d1[0.365] d2[0.265] g[209.785]
>18, d1[0.290] d2[0.230] g[177.528]
>19, d1[0.213] d2[0.112] g[212.175]
>20, d1[0.283] d2[0.053] g[202.634]
>21, d1[0.245] d2[0.013] g[198.915]
>22, d1[0.249] d2[0.017] g[165.955]
>23, d1[0.265] d2[0.049] g[187.037]
>24, d1[0.252] d2[0.014] g[161.306]
>25, d1[0.278] d2[0.010] g[183.348]
>26, d1[0.250] d2[0.018] g[186.888]
>27, d1[0.184] d2[0.044] g[175.365]
>28, d1[0.822] d2[1.371] g[215.129]
>

>228, d1[0.302] d2[0.326] g[93.335]
>229, d1[0.329] d2[0.371] g[66.725]
>230, d1[0.373] d2[0.380] g[60.978]
>231, d1[0.315] d2[0.352] g[99.363]
>232, d1[0.296] d2[0.337] g[100.897]
>233, d1[0.346] d2[0.334] g[75.194]
>234, d1[0.295] d2[0.378] g[71.494]
>235, d1[0.380] d2[0.310] g[75.443]
>236, d1[0.445] d2[0.347] g[70.460]
>237, d1[0.314] d2[0.367] g[80.993]
>238, d1[0.362] d2[0.321] g[73.990]
>239, d1[0.318] d2[0.301] g[108.351]
>240, d1[0.317] d2[0.327] g[84.652]
>241, d1[0.355] d2[0.338] g[70.603]
>242, d1[0.284] d2[0.286] g[69.187]
>243, d1[0.293] d2[0.271] g[67.112]
>244, d1[0.289] d2[0.247] g[89.668]
>245, d1[0.301] d2[0.309] g[73.158]
>246, d1[0.375] d2[0.311] g[76.237]
>247, d1[0.299] d2[0.177] g[73.304]
>248, d1[0.509] d2[0.160] g[86.681]
>249, d1[0.304] d2[0.170] g[87.676]
>250, d1[0.315] d2[0.106] g[78.614]
>251, d1[0.284] d2[0.126] g[91.356]
>252, d1[0.299] d2[0.062] g[77.835]
>253, d1[1.294] d2[0.214] g[70.421]
>254, d1[0.285] d2[0.282] g[66.443]
>255, d1[0.304] d2[0.257] 

>456, d1[0.413] d2[0.362] g[62.246]
>457, d1[0.274] d2[0.120] g[79.140]
>458, d1[0.404] d2[0.429] g[51.169]
>459, d1[0.327] d2[0.107] g[67.089]
>460, d1[0.322] d2[0.280] g[50.442]
>461, d1[0.353] d2[0.342] g[60.007]
>462, d1[0.397] d2[0.328] g[51.074]
>463, d1[0.309] d2[0.453] g[62.469]
>464, d1[0.754] d2[0.191] g[60.377]
>465, d1[0.593] d2[0.151] g[61.537]
>466, d1[0.251] d2[0.479] g[64.922]
>467, d1[0.253] d2[0.260] g[66.673]
>468, d1[0.499] d2[0.223] g[56.462]
>469, d1[0.320] d2[0.480] g[59.473]
>470, d1[0.756] d2[0.255] g[54.265]
>471, d1[0.262] d2[0.470] g[55.550]
>472, d1[0.253] d2[0.259] g[55.222]
>473, d1[0.441] d2[0.260] g[57.709]
>474, d1[0.302] d2[0.331] g[52.984]
>475, d1[0.278] d2[0.396] g[51.706]
>476, d1[0.444] d2[0.246] g[60.502]
>477, d1[0.306] d2[0.137] g[53.839]
>478, d1[0.348] d2[0.412] g[61.741]
>479, d1[0.243] d2[0.203] g[70.458]
>480, d1[0.274] d2[0.258] g[66.911]
>481, d1[0.272] d2[0.515] g[68.942]
>482, d1[0.305] d2[0.443] g[44.444]
>483, d1[0.276] d2[0.387] g[

>684, d1[0.872] d2[0.171] g[42.043]
>685, d1[0.210] d2[0.279] g[52.700]
>686, d1[0.578] d2[0.199] g[41.438]
>687, d1[0.508] d2[0.275] g[46.976]
>688, d1[0.251] d2[0.232] g[51.158]
>689, d1[0.176] d2[0.261] g[51.312]
>690, d1[0.469] d2[0.205] g[51.308]
>691, d1[0.251] d2[0.258] g[44.874]
>692, d1[0.290] d2[0.345] g[45.792]
>693, d1[0.351] d2[0.354] g[50.130]
>694, d1[0.178] d2[0.523] g[42.906]
>695, d1[0.699] d2[0.177] g[49.505]
>696, d1[0.598] d2[0.198] g[47.097]
>697, d1[0.368] d2[0.263] g[61.265]
>698, d1[0.536] d2[0.257] g[54.404]
>699, d1[0.373] d2[0.215] g[48.420]
>700, d1[0.420] d2[0.342] g[39.262]
>701, d1[0.346] d2[0.331] g[53.502]
>702, d1[0.426] d2[0.288] g[50.669]
>703, d1[0.403] d2[0.311] g[50.951]
>704, d1[0.230] d2[0.410] g[50.136]
>705, d1[0.106] d2[0.395] g[49.576]
>706, d1[0.394] d2[0.217] g[44.638]
>707, d1[0.220] d2[0.252] g[41.502]
>708, d1[0.107] d2[0.258] g[52.444]
>709, d1[0.336] d2[0.237] g[53.606]
>710, d1[0.582] d2[0.279] g[63.667]
>711, d1[0.310] d2[0.215] g[

>912, d1[0.274] d2[0.306] g[40.045]
>913, d1[0.301] d2[0.277] g[35.543]
>914, d1[0.550] d2[0.249] g[32.991]
>915, d1[0.421] d2[0.183] g[43.498]
>916, d1[0.487] d2[0.253] g[33.018]
>917, d1[0.259] d2[0.329] g[45.360]
>918, d1[0.279] d2[0.331] g[36.303]
>919, d1[0.503] d2[0.216] g[40.169]
>920, d1[0.342] d2[0.224] g[36.060]
>921, d1[0.405] d2[0.348] g[33.759]
>922, d1[0.299] d2[0.145] g[44.087]
>923, d1[0.521] d2[0.471] g[34.402]
>924, d1[0.226] d2[0.249] g[47.816]
>925, d1[0.764] d2[0.186] g[45.001]
>926, d1[0.439] d2[0.288] g[44.344]
>927, d1[0.531] d2[0.274] g[37.590]
>928, d1[0.374] d2[0.427] g[35.514]
>929, d1[0.308] d2[0.347] g[38.608]
>930, d1[0.190] d2[0.487] g[43.765]
>931, d1[0.295] d2[0.228] g[32.290]
>932, d1[0.236] d2[0.213] g[39.122]
>933, d1[0.440] d2[0.249] g[40.293]
>934, d1[0.513] d2[0.301] g[36.176]
>935, d1[0.301] d2[0.270] g[38.058]
>936, d1[0.354] d2[0.357] g[37.245]
>937, d1[0.313] d2[0.375] g[40.043]
>938, d1[0.418] d2[0.301] g[47.460]
>939, d1[0.449] d2[0.295] g[

...

>54973, d1[0.348] d2[0.345] g[3.929]
>54974, d1[0.348] d2[0.348] g[3.517]
>54975, d1[0.348] d2[0.347] g[3.131]
>54976, d1[0.347] d2[0.349] g[3.691]
>54977, d1[0.344] d2[0.349] g[3.799]
>54978, d1[0.349] d2[0.347] g[4.285]
>54979, d1[0.347] d2[0.351] g[4.461]
>54980, d1[0.347] d2[0.349] g[4.077]
>54981, d1[0.345] d2[0.347] g[3.679]
>54982, d1[0.347] d2[0.344] g[3.734]
>54983, d1[0.351] d2[0.348] g[3.879]
>54984, d1[0.349] d2[0.342] g[3.700]
>54985, d1[0.348] d2[0.346] g[4.080]
>54986, d1[0.344] d2[0.344] g[3.650]
>54987, d1[0.349] d2[0.346] g[4.566]
>54988, d1[0.355] d2[0.349] g[4.047]
>54989, d1[0.346] d2[0.348] g[4.248]
>54990, d1[0.344] d2[0.350] g[3.827]
>54991, d1[0.348] d2[0.345] g[4.355]
>54992, d1[0.347] d2[0.345] g[4.134]
>54993, d1[0.348] d2[0.351] g[3.897]
>54994, d1[0.342] d2[0.351] g[3.890]
>54995, d1[0.348] d2[0.344] g[3.494]
>54996, d1[0.344] d2[0.343] g[3.902]
>54997, d1[0.348] d2[0.347] g[3.678]
>54998, d1[0.349] d2[0.348] g[3.776]
>54999, d1[0.348] d2[0.350] g[3.922]
>

>55193, d1[0.351] d2[0.346] g[4.002]
>55194, d1[0.347] d2[0.347] g[4.139]
>55195, d1[0.346] d2[0.348] g[3.981]
>55196, d1[0.345] d2[0.348] g[3.957]
>55197, d1[0.349] d2[0.347] g[3.519]
>55198, d1[0.343] d2[0.346] g[4.796]
>55199, d1[0.350] d2[0.350] g[3.711]
>55200, d1[0.345] d2[0.352] g[3.634]
>55201, d1[0.352] d2[0.348] g[3.647]
>55202, d1[0.345] d2[0.347] g[3.738]
>55203, d1[0.346] d2[0.348] g[3.787]
>55204, d1[0.344] d2[0.345] g[3.996]
>55205, d1[0.348] d2[0.349] g[3.741]
>55206, d1[0.349] d2[0.346] g[3.863]
>55207, d1[0.352] d2[0.347] g[3.687]
>55208, d1[0.347] d2[0.351] g[3.576]
>55209, d1[0.347] d2[0.350] g[3.733]
>55210, d1[0.348] d2[0.343] g[4.030]
>55211, d1[0.352] d2[0.341] g[4.964]
>55212, d1[0.347] d2[0.347] g[4.386]
>55213, d1[0.349] d2[0.346] g[4.456]
>55214, d1[0.348] d2[0.349] g[3.797]
>55215, d1[0.341] d2[0.344] g[3.699]
>55216, d1[0.349] d2[0.343] g[3.866]
>55217, d1[0.344] d2[0.343] g[3.547]
>55218, d1[0.349] d2[0.342] g[3.678]
>55219, d1[0.349] d2[0.353] g[4.090]
>

>55415, d1[0.346] d2[0.346] g[3.460]
>55416, d1[0.349] d2[0.346] g[3.901]
>55417, d1[0.349] d2[0.349] g[3.670]
>55418, d1[0.348] d2[0.352] g[3.684]
>55419, d1[0.352] d2[0.346] g[3.953]
>55420, d1[0.350] d2[0.346] g[3.785]
>55421, d1[0.346] d2[0.345] g[3.981]
>55422, d1[0.342] d2[0.344] g[4.161]
>55423, d1[0.347] d2[0.346] g[3.907]
>55424, d1[0.350] d2[0.352] g[3.941]
>55425, d1[0.352] d2[0.351] g[4.049]
>55426, d1[0.349] d2[0.349] g[3.922]
>55427, d1[0.349] d2[0.348] g[4.475]
>55428, d1[0.348] d2[0.345] g[4.107]
>55429, d1[0.349] d2[0.343] g[3.539]
>55430, d1[0.346] d2[0.351] g[3.622]
>55431, d1[0.347] d2[0.347] g[4.821]
>55432, d1[0.344] d2[0.345] g[3.888]
>55433, d1[0.346] d2[0.343] g[4.613]
>55434, d1[0.349] d2[0.345] g[4.053]
>55435, d1[0.344] d2[0.345] g[4.635]
>55436, d1[0.343] d2[0.345] g[5.003]
>55437, d1[0.347] d2[0.351] g[4.353]
>55438, d1[0.348] d2[0.348] g[3.619]
>55439, d1[0.346] d2[0.343] g[4.541]
>55440, d1[0.349] d2[0.343] g[3.760]
>55441, d1[0.349] d2[0.350] g[3.943]
>

>55637, d1[0.349] d2[0.347] g[4.112]
>55638, d1[0.348] d2[0.344] g[3.737]
>55639, d1[0.345] d2[0.347] g[3.565]
>55640, d1[0.346] d2[0.350] g[3.373]
>55641, d1[0.346] d2[0.347] g[3.898]
>55642, d1[0.346] d2[0.343] g[3.607]
>55643, d1[0.350] d2[0.350] g[3.276]
>55644, d1[0.344] d2[0.351] g[3.612]
>55645, d1[0.339] d2[0.348] g[3.813]
>55646, d1[0.345] d2[0.344] g[3.574]
>55647, d1[0.346] d2[0.348] g[4.633]
>55648, d1[0.345] d2[0.349] g[4.321]
>55649, d1[0.348] d2[0.344] g[3.651]
>55650, d1[0.352] d2[0.345] g[3.406]
>55651, d1[0.345] d2[0.353] g[3.777]
>55652, d1[0.348] d2[0.343] g[3.534]
>55653, d1[0.343] d2[0.345] g[4.131]
>55654, d1[0.349] d2[0.351] g[3.845]
>55655, d1[0.356] d2[0.350] g[3.771]
>55656, d1[0.342] d2[0.348] g[3.529]
>55657, d1[0.351] d2[0.348] g[3.627]
>55658, d1[0.348] d2[0.349] g[4.132]
>55659, d1[0.347] d2[0.345] g[4.997]
>55660, d1[0.348] d2[0.348] g[4.111]
>55661, d1[0.345] d2[0.349] g[3.696]
>55662, d1[0.349] d2[0.352] g[3.520]
>55663, d1[0.345] d2[0.340] g[3.819]
>

>55859, d1[0.341] d2[0.353] g[3.692]
>55860, d1[0.344] d2[0.346] g[3.667]
>55861, d1[0.348] d2[0.347] g[4.057]
>55862, d1[0.347] d2[0.345] g[4.294]
>55863, d1[0.344] d2[0.346] g[4.124]
>55864, d1[0.345] d2[0.348] g[3.798]
>55865, d1[0.342] d2[0.350] g[3.528]
>55866, d1[0.347] d2[0.344] g[4.564]
>55867, d1[0.348] d2[0.344] g[3.818]
>55868, d1[0.344] d2[0.350] g[4.192]
>55869, d1[0.346] d2[0.349] g[3.995]
>55870, d1[0.343] d2[0.350] g[3.503]
>55871, d1[0.346] d2[0.351] g[3.662]
>55872, d1[0.345] d2[0.349] g[4.052]
>55873, d1[0.349] d2[0.353] g[3.778]
>55874, d1[0.344] d2[0.351] g[4.244]
>55875, d1[0.345] d2[0.350] g[3.949]
>55876, d1[0.348] d2[0.352] g[3.628]
>55877, d1[0.345] d2[0.347] g[3.323]
>55878, d1[0.352] d2[0.346] g[3.729]
>55879, d1[0.345] d2[0.354] g[3.868]
>55880, d1[0.350] d2[0.347] g[3.764]
>55881, d1[0.343] d2[0.351] g[3.945]
>55882, d1[0.350] d2[0.350] g[3.993]
>55883, d1[0.349] d2[0.344] g[4.519]
>55884, d1[0.350] d2[0.353] g[4.163]
>55885, d1[0.353] d2[0.345] g[3.904]
>

In [19]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()
#end the tensorflow session

In [20]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()

# 